I refered the kernel(https://www.kaggle.com/rohanrao/ashrae-half-and-half), Thank you Vopani!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/"directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import gc
import os
import random

import lightgbm as lgb
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

path_data = "/kaggle/input/ashrae-energy-prediction/"
path_train = path_data + "train.csv"
path_test = path_data + "test.csv"
path_building = path_data + "building_metadata.csv"
path_weather_train = path_data + "weather_train.csv"
path_weather_test = path_data + "weather_test.csv"

In [ ]:
train_df = pd.read_csv(path_train)

building_df = pd.read_csv(path_building)
le = LabelEncoder()
building_df.primary_use = le.fit_transform(building_df.primary_use)

weather_train_df = pd.read_csv(path_weather_train)

# Preparing data

In [ ]:
#reduce the memory usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

train_df = reduce_mem_usage(train_df)
building_df = reduce_mem_usage(building_df)
weather_train_df = reduce_mem_usage(weather_train_df)

In [ ]:
def preparing_data(X, bld_data, wth_data, test=False):
    #merge three data
    X = X.merge(bld_data, on = "building_id", how = "left")
    X = X.merge(wth_data, on = ["site_id", "timestamp"], how = "left")
    
    #change the data type
    X["timestamp"] = pd.to_datetime(X["timestamp"], format="%Y-%m-%d %H:%M:%S")
    #scale down the square feet
    X["square_feet"] = np.log1p(X["square_feet"])
    
    if not test:
        X.sort_values("timestamp", inplace = True)
        X.reset_index(drop=True, inplace = True)
    
    #separte timestamp into hour and weekday
    X["hour"] = X.timestamp.dt.hour
    X["weekday"] = X.timestamp.dt.weekday
    
    # add new feature "isholiday"
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04","2016-09-05", "2016-10-10",
                "2016-11-11", "2016-11-24", "2016-12-26", "2017-01-01", "2017-01-16", "2017-02-20", "2017-05-29",
                "2017-07-04", "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25", "2018-01-01",
                "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04", "2018-09-03", "2018-10-08", "2018-11-12",
                "2018-11-22", "2018-12-25", "2019-01-01"]
    X["isholiday"] = (X.timestamp.dt.date.astype("str").isin(holidays)).astype(int)
    
    X.drop("timestamp", axis=1, inplace=True)
    
    if test:
        row_ids = X.row_id
        X.drop("row_id", axis=1, inplace=True)
        return X, row_ids
    else:
        y = np.log1p(X.meter_reading)
        X.drop("meter_reading", axis=1, inplace=True)
        return X, y

In [ ]:
X_full, y_full = preparing_data(train_df, building_df, weather_train_df)

del train_df, weather_train_df
gc.collect()

# Read data

In [ ]:
# print(X_full.shape)
# X_full.head()

In [ ]:
for i in X_full.columns:
    print(i, X_full[i].dtype)

# Creating model
*Half and Half LGBM model*

In [ ]:
categorical_features = ["building_id", "site_id", "meter", "primary_use", "hour", "weekday", "isholiday"]

X_half1 = X_full.iloc[:int(len(X_full)/2)]
X_half2 = X_full[int(len(X_full)/2):]
y_half1 = y_full[:int(len(X_full)/2)]
y_half2 = y_full[int(len(X_full)/2):]

lgb_train1_data = lgb.Dataset(X_half1, label=y_half1, categorical_feature=categorical_features, free_raw_data=False)
lgb_train2_data = lgb.Dataset(X_half2, label=y_half2, categorical_feature=categorical_features, free_raw_data=False)

parameters = {
    "objective": "regression",
    "boosting": "gbdt",
    "num_leaves": 100,
    "learning_rate": 0.05,
    "feature_fraction": 0.80,
    "reg_lambda": 2,
    "metric": "rmse",
    "n_jobs": 4
}

print("start building model1_1 : ")
model1_1 = lgb.train(parameters, train_set=lgb_train1_data, num_boost_round=1000, valid_sets = [lgb_train1_data, lgb_train2_data],
                  verbose_eval=200, early_stopping_rounds=200)
print("start building model1_2 : ")
model1_2 = lgb.train(parameters, train_set=lgb_train2_data, num_boost_round=1000, valid_sets=[lgb_train2_data, lgb_train1_data],
                    verbose_eval=200, early_stopping_rounds=200)

In [ ]:
del X_half1, X_half2, y_half1, y_half2, lgb_train1_data, lgb_train2_data, categorical_features
gc.collect()

# Feature Importance

In [ ]:
df1_feature_imp = pd.DataFrame()
df1_feature_imp["feature"] = X_full.columns.values
df1_feature_imp["importance"] = model1_1.feature_importance()
df2_feature_imp = pd.DataFrame()
df2_feature_imp["feature"] = X_full.columns.values
df2_feature_imp["importance"] = model1_2.feature_importance()

df_ft_imp = pd.concat([df1_feature_imp, df2_feature_imp], axis=0)


plt.figure(figsize=(16, 10))
sns.barplot(x = "importance", y="feature", data = df_ft_imp.sort_values(by="importance", ascending=False))
plt.title("Light GBM Feature Importance")

# preparing test data

In [ ]:
test_full = pd.read_csv(path_test)
weather_test = pd.read_csv(path_weather_test)

test_full = reduce_mem_usage(test_full)
weather_test = reduce_mem_usage(weather_test)

X_test, row_ids = preparing_data(test_full, building_df, weather_test, test=True)

In [ ]:
del test_full, building_df, weather_test
gc.collect()

# scoring test data

In [ ]:
predict = np.expm1(model1_1.predict(X_test, num_iteration = model1_1.best_iteration)) / 2

del model1_1
gc.collect()

predict += np.expm1(model1_2.predict(X_test, num_iteration = model1_2.best_iteration)) / 2
del model1_2
gc.collect()

### submission

In [ ]:
submission = pd.DataFrame({"row_id": row_ids, "meter_reading": np.clip(predict, 0, a_max=None)})
submission.to_csv("submission.csv", index=False)
# del submission, predict
# gc.collect()

In [ ]:
submission.shape